In [ ]:
import pymongo
from pprint import pprint
from pymongo import MongoClient 
from bson.son import SON
myclient = MongoClient('localhost', 27017)
horizondb = myclient['horizondb3']
things = horizondb["things"]
things.insert_one({"x": 1, "tags": ["dog", "cat"]})
things.insert_one({"x": 2, "tags": ["cat"]})
things.insert_one({"x": 2, "tags": ["mouse", "cat", "dog"]})
things.insert_one({"x": 3, "tags": []})

In [ ]:
results = things.aggregate([ {"$unwind": "$tags"},
                  {"$group": {"_id": "$tags", "count": {"$sum": 1}}},
                  {"$sort": SON([("count", -1), ("_id", -1)])}
                  ])

In [ ]:
for result in results:
    print(result)

## MaprReduce

In [ ]:
from bson.code import Code
mapper = Code(""" function () {
              this.tags.forEach(function(z) {
                  emit(z, 1)
              });
              }
               """)
reducer = Code(""" function (key, values) {
                         var total = 0;
                         for (var i = 0; i < values.length; i++) {
                         total += values[i];
                         }
                         return total;
                    }
                    """)

results = things.map_reduce(mapper, reducer, "myresults")
for doc in results.find():
    print(doc)

## Advanced Map/Reduce

In [ ]:
results = things.map_reduce(mapper, reducer, "myresults", query={"x": {"$lt": 2}})
for doc in results.find():
    print(doc)